In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
from keras.models import Model
from keras.layers import Dense, Flatten, BatchNormalization, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import resample
import tensorflow_addons as tfa

#Đường dẫn thư mục train và test
train_path = r'.\train'
test_path = r'.\test'
val_path = r'.\val'

batch_size = 16 

img_height = 500
img_width = 500

c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8,1.2]
)

# Data augmentation for testing/validation
test_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = image_gen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

# Testing data generator
test = test_data_gen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='rgb', 
    shuffle=False,
    class_mode='binary',
    batch_size=batch_size
)

# Validation data generator
validation_generator = image_gen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

Found 1943 images belonging to 2 classes.
Found 611 images belonging to 2 classes.
Found 497 images belonging to 2 classes.


In [5]:
X_train, y_train = [], []
for _ in range(train_generator.n // batch_size + 1):
    x, y = train_generator.__next__()
    X_train.append(x)
    y_train.append(y)

X_train = np.vstack(X_train)
y_train = np.hstack(y_train)

# Separate majority and minority classes
normal = X_train[y_train == 0]
disease = X_train[y_train == 1]

# Oversample minority class
normal_upsampled = resample(normal, 
                            replace=True,     # sample with replacement
                            n_samples=len(disease),    # to match majority class
                            random_state=123) # reproducible results

# Combine majority class with upsampled minority class
X_train_balanced = np.vstack([normal_upsampled, disease])
y_train_balanced = np.hstack([np.zeros(len(disease)), np.ones(len(disease))])

# Shuffle the dataset
indices = np.arange(X_train_balanced.shape[0])
np.random.shuffle(indices)
X_train_balanced = X_train_balanced[indices]
y_train_balanced = y_train_balanced[indices]

In [6]:
# Define the DenseNet model
base_model = DenseNet121(include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers[:-10]:  # Fine-tune the last 10 layers
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout layer
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout layer
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
cnn = Model(inputs=base_model.input, outputs=predictions)
cnn.compile(optimizer='adam', loss=tfa.losses.SigmoidFocalCrossEntropy(), metrics=['accuracy'])

# Callbacks
early = EarlyStopping(monitor="val_loss", mode="min", patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.2, min_lr=0.000001)
callbacks_list = [early, learning_rate_reduction]

# Train the model
cnn.fit(
    X_train_balanced, y_train_balanced,
    epochs=30,  # Increase the number of epochs
    validation_data=validation_generator,
    callbacks=callbacks_list,
    batch_size=batch_size
)


Epoch 1/30
174/174 [==============================] - 1493s 9s/step - loss: 0.1981 - accuracy: 0.5672 - val_loss: 0.0765 - val_accuracy: 0.4608 - lr: 0.0010
Epoch 2/30
174/174 [==============================] - 750s 4s/step - loss: 0.1144 - accuracy: 0.6207 - val_loss: 0.0650 - val_accuracy: 0.6358 - lr: 0.0010
Epoch 3/30
174/174 [==============================] - 485s 3s/step - loss: 0.0843 - accuracy: 0.6613 - val_loss: 0.0765 - val_accuracy: 0.5533 - lr: 0.0010
Epoch 4/30
174/174 [==============================] - 478s 3s/step - loss: 0.0639 - accuracy: 0.7238 - val_loss: 0.0507 - val_accuracy: 0.7565 - lr: 0.0010
Epoch 5/30
174/174 [==============================] - 480s 3s/step - loss: 0.0456 - accuracy: 0.7981 - val_loss: 0.0694 - val_accuracy: 0.6278 - lr: 0.0010
Epoch 6/30
174/174 [==============================] - 476s 3s/step - loss: 0.0358 - accuracy: 0.8391 - val_loss: 0.0603 - val_accuracy: 0.7425 - lr: 0.0010
Epoch 7/30
174/174 [==============================] - 473s 3s/s

In [7]:
cnn.save('DenseNet3.h5')

c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate the model on the test data
test_accu = cnn.evaluate(test)
print('The testing accuracy is:', test_accu[1] * 100, '%')

# Predict the labels for the test data
y_pred = cnn.predict(test)
y_pred = np.round(y_pred).astype(int)  # Chuyển đổi dự đoán thành nhãn nhị phân (0 hoặc 1)

# Get the true labels
y_true = test.classes

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Compute classification report
class_report = classification_report(y_true, y_pred, target_names=['Disease', 'Normal'])
print('Classification Report:')
print(class_report)

# Extract Sensitivity, Specificity, and F1-Score from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity  # Recall là tên khác của Sensitivity
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Sensitivity: {sensitivity * 100:.2f}%')
print(f'Specificity: {specificity * 100:.2f}%')
print(f'F1-Score: {f1_score:.2f}')

39/39 [==============================] - 86s 2s/step - loss: 0.0814 - accuracy: 0.7905
The testing accuracy is: 79.05073761940002 %
39/39 [==============================] - 88s 2s/step
Confusion Matrix:
[[159  38]
 [ 90 324]]
Classification Report:
              precision    recall  f1-score   support

     Disease       0.64      0.81      0.71       197
      Normal       0.90      0.78      0.84       414

    accuracy                           0.79       611
   macro avg       0.77      0.79      0.77       611
weighted avg       0.81      0.79      0.80       611

Sensitivity: 78.26%
Specificity: 80.71%
F1-Score: 0.84
